# Checking GPU details

In [1]:
!nvidia-smi

Wed Jan 24 17:55:41 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        Off | 00000000:01:00.0 Off |                  Off |
|  0%   28C    P8              28W / 270W |     11MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Importing necessary modules

In [2]:
import os
import re
import time
import copy
import json
import random
import pickle
import subprocess

from datetime import datetime
from collections import Counter

import numpy as np
import pandas as pd
from tqdm import tqdm

import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.font_manager import FontProperties

import torch
import torch.nn as nn
import torch.functional as F
from torch import autocast
from torch.cuda.amp import GradScaler 
from torch.utils.data import Dataset, DataLoader

SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Handling hyper-parameters JSON file

In [3]:
json_file = "./utils/hyperparameters.json"    # JSON file path

def get_parm(key):
    """
       function will take the key as input and return the corresponding value from the json file.
       Input  : "wave_pickle_path" 
       Output : "/workspace/Machine Translation Code/Dataset/audio/audio_data.pickle"
    """
    # Reading JSON file
    with open(json_file,'r') as f:
        parm_dict = json.load(f)       
    return parm_dict[key]            

def put_parm(key, value):
    """
       function will take the key and value as input and will save the key, value in the json file.
       Input  : "tgt_max_sent_len", 11
    """
    # Reading JSON file
    with open(json_file,'r') as f:
        parm_dict = json.load(f)

    parm_dict[key] = value          # Adding key, value to the json dict
    
    # Saving JSON file
    with open(json_file,'w') as f:
        json.dump(parm_dict, f)


# Checking available device(cpu or gpu)

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
put_parm("device", device)                  # Adding available device to JSON file
print("Available Device : ", device)

Available Device :  cuda


# Loading Dataset

In [5]:
class Read_Dataset():
    def __init__(self):
        """
            Dataset from the CSV file will be loaded and saved partition-wise in different DataFrames.
            Source and target dictionary objects will be added
        """
        
        data_path = get_parm("dataset_csv")            # Dataset file path
            
        self.df = pd.read_csv(data_path, usecols = ['image_id','P_TYPE', 'English Text Pre','Hindi Text Pre'])
        self.df_train = self.df[self.df['P_TYPE'] == 'train']          # Training Set
        self.df_train = self.df_train.drop_duplicates()                # Removing duplicate samples
        self.df_val   = self.df[self.df['P_TYPE'] == 'val']            # Validation Set
        self.df_val   = self.df_val.drop_duplicates()                  # Removing duplicate samples
        self.df_test  = self.df[self.df['P_TYPE'] == 'test']           # Test Set
        self.df_test  = self.df_test.drop_duplicates()                 # Removing duplicate samples
        self.df_challenge = self.df[self.df['P_TYPE'] == 'challenge']  # Challenge Set
        self.df_challenge = self.df_challenge.drop_duplicates()        # Removing duplicate samples
        self.src_vocab = None                                          # Source Vocabulary
        self.tgt_vocab = None                                          # Target Vocabulary
        print("Train : {}, Val : {}, Test : {}, Challenge : {}".format(len(self.df_train), len(self.df_val), len(self.df_test), len(self.df_challenge)))
 
    def output_to_sentence_val(self, idx_list, src, outputs, p_type):
        """
           function takes the index list of sentences, source(model_input), output sentences and partition type as input
           and return the original source English, hindi, model input and generated sentences as list
        """
        
        if p_type == 'val':                 #
            df = self.df_val                #
        elif p_type == 'test':              #  Select input partition's DataFrame
            df = self.df_test               #
        elif p_type == 'challenge':         #
            df = self.df_challenge          #
            
        #since = time.time()
        src_english = []                    # Source Original English Sentence
        inp_english = []                    # Source Model Input English Sentence
        tgt_hindi = []                      # Source Original Hindi Sentence
        pred_hindi = []                      # Output Hindi Sentences
        
        # Iterating through each indexed sentences
        for i in range(len(idx_list)):
            idx = idx_list[i].item()                                     # Get Index value from tensor
            src_english.append(df['English Text Pre'].iloc[idx] +'\n')   # Getting Original Source English sentence for the given indedx
            tgt_hindi.append(df['Hindi Text Pre'].iloc[idx] +'\n')       # Getting Original Source Hindi sentence for the given indedx

            # ********** Getting Model input Sentences ***************
            inp_sent = []                     # Will contain model input sentence word
            inp_seq = src[i]                  # Current model input English sentence
            
            # Iterating through each input word 
            for word_idx in inp_seq[:]:
                word_idx = word_idx.item()    # Get word index value from tensor
                if word_idx == 0:             # Stop if <PAD> index 
                    break
                else:
                    word = self.src_vocab.itos.get(word_idx, '<UNK>')    # Get word of current word index
                    inp_sent.append(word)  
            inp_english.append(' '.join(inp_sent) +'\n')   # Word list to sentence string
            
            # ********** Getting Model Output Sentences ***************
            tgt_sent = []                                    # Will contain generated sentence's words
            pred_seq = torch.argmax(outputs[i], dim = 1)     # Predicted Sentence (contains word indexes)    
            # Iterating through each generated word 
            for word_idx in pred_seq[1:]:
                word_idx = word_idx.item()    # Get word index value from tensor
                word = self.tgt_vocab.itos.get(word_idx, '<UNK>')
                if word_idx == 2:             # Stop if <eos> index 
                    break
                else:
                    tgt_sent.append(word)      
            pred_hindi.append(' '.join(tgt_sent) +'\n')   # Word list to sentence string
            i += 1
            
        #time_elapsed = time.time() - since 
        #print('Validation Completed in {}'.format(time_elapsed))
        return src_english, inp_english, tgt_hindi, pred_hindi
        
mt_data = Read_Dataset()
mt_data.df       

Train : 28919, Val : 998, Test : 1595, Challenge : 1400


,image_id,P_TYPE,English Text Pre,Hindi Text Pre
0,11,train,it is an indoor scene,यह एक इनडोर दृश्य है
1,17,train,computer screens turned on,कंप्यूटर स्क्रीन चालू
2,18,train,man has short hair,आदमी के छोटे बाल हैं
3,21,train,photo album open on an adult lap,एक वयस्क की गोद में फोटो एल्बम खुला
4,23,train,there is a group of girls beside the black car,काली कार के पास लड़कियों का एक समूह है
...,...,...,...,...
32907,2358720,challenge,a fence between the field and players,मैदान और खिलाड़ियों के बीच एक बाड़
32908,2414143,challenge,sponsor signs hanging on the walls of the tenn...,टेनिस कोर्ट की दीवारों पर प्रायोजक के निशान लट...
32909,2941,challenge,many stairs up to a second story,कई सीढ़ियाँ एक दूसरी कहानी तक
32910,2334985,challenge,section of fine gray sand,महीन भूरे बालू का खंड


# Building Vocabulary

In [6]:
class Vocabulary:
    def __init__(self, word_freq_thresh=0, max_sent_len = 10):
        self.itos = {0:'<PAD>', 1:"<SOS>", 2:"<EOS>", 3:"<UNK>"}     # Default Vocab dict {index : word}
        self.stoi = {s:i for i,s in self.itos.items()}               # Default Vocab dict {word : index}
        self.word_freq_thresh = word_freq_thresh      # A word should present >= threshold times in the corpus
        self.max_sent_len = max_sent_len              # Maximum length of a sentence
        self.vocab_size = len(self.itos)              # Size of Vocabulary
        
    def truncate_sentence(self, sentence_list):
        trunc_sentences = []                          # Truncated sentences will be added to the list
        # Iterating through each input sentence
        for sent in sentence_list:       
            trunc_sentences.append(' '.join(sent.split()[:self.max_sent_len]))   # Truncating sentence after the defined length
        return trunc_sentences
    
    def build_vocabulary(self, sentence_list):
        word_list = []                                           # Will contain each sentence's word 
        trunc_sentences = self.truncate_sentence(sentence_list)  # Truncate words after specified length
        for sent in trunc_sentences:       
            word_list.extend(sent.split())                       # Getting words of truncated sentences
        word_freq = Counter(word_list)                           # Getting each word's frequency        
        word_freq = sorted(word_freq.items(), key=lambda x:x[1], reverse=True)  # Sorting word frequency dict 
        idx = 4                                       # Index will start from 4
        total_tokens = 0
        
        # Assigning index to each word crossing the frequency threshold
        for word, freq in word_freq:
            if freq >= self.word_freq_thresh:         # Check for minimum word frequency threshold
                self.itos[idx] = word                 # Assign index to word
                idx += 1
                total_tokens += freq
            else:
                pass
                
        self.stoi = {s:i for i,s in self.itos.items()}     # Updating word to index dictionary
        self.vocab_size = len(self.itos)                   # Vocabulary Size
        
        print("Total Tokens in the Dataset : ",total_tokens)
        print("Total Unique Tokens(Except pre-defined tokens) : ", self.vocab_size - 4)
        print(self.itos)
    
    def sentence_indexing(self, sentence_list):
        no_of_sent = len(sentence_list)                           # No of sentences in the dataset
        output = torch.zeros(no_of_sent, self.max_sent_len+2)     # Output indexed tensor
        sos_idx = self.stoi.get('<SOS>')                          # Getting <SOS> index in vocabulary
        eos_idx = self.stoi.get('<EOS>')                          # Getting <EOS> index in vocabulary
        unk_idx = self.stoi.get('<UNK>')                          # Getting <UNK> index in vocabulary
        
        # Iterating through each sentence
        for i in range(no_of_sent):                         
            sent_idx = torch.zeros(self.max_sent_len+2)           # Initial empty tensor for curent sentence
            sent_idx[0] = sos_idx                                 # Add <SOS> index to first index
            word_list = sentence_list[i].split()                  # Split sentence into words
            sent_len = len(word_list)                             # Length of the sentence i.e. no of words

            # Iterating through each word in sentence (will skip extra words after maximum length)
            for j in range(1, min(sent_len,self.max_sent_len) + 1):
                sent_idx[j] = (self.stoi.get(word_list[j-1],unk_idx))   # Get word index from Vocab, append <UNK> index if not found in vocabulary
            sent_idx[j+1] = eos_idx                               # Add <EOS> index
#             print("Sentence : ", sentence_list[i])
#             print("Indexing : ", sent_idx)
            output[i,:] = sent_idx                                # Adding current sentence index tensor to ouput tensor
        
        return output

### Vocabulary Hyper-Parameters


In [7]:
src_min_word_freq = get_parm("src_min_word_freq")
src_max_sent_len = get_parm("src_max_sent_len")  
tgt_min_word_freq = get_parm("tgt_min_word_freq") 
tgt_max_sent_len = get_parm("tgt_max_sent_len")

print("Minimum Source Sentence Frequency : ", src_min_word_freq)
print("Minimum Target Sentence Frequency : ", tgt_min_word_freq)
print("Maximum Source Sentence Length : ", src_max_sent_len)
print("Maximum Target Sentence Length : ", tgt_max_sent_len)

Minimum Source Sentence Frequency :  3
Minimum Target Sentence Frequency :  3
Maximum Source Sentence Length :  10
Maximum Target Sentence Length :  10


### Build src vocab

In [8]:
src_vocab = Vocabulary(src_min_word_freq, src_max_sent_len)          # Default Vocab
src_vocab.build_vocabulary(mt_data.df_train['English Text Pre'])     # Building Vocabulary
mt_data.src_vocab = src_vocab
put_parm("src_vocab_size", src_vocab.vocab_size)

Total Tokens in the Dataset :  138896
Total Unique Tokens(Except pre-defined tokens) :  2210
{0: '<PAD>', 1: '<SOS>', 2: '<EOS>', 3: '<UNK>', 4: 'a', 5: 'the', 6: 'on', 7: 'of', 8: 'is', 9: 'in', 10: 'white', 11: 'man', 12: 'and', 13: 'black', 14: 'with', 15: 'red', 16: 'blue', 17: 'this', 18: 'woman', 19: 'green', 20: 'person', 21: 'brown', 22: 'two', 23: 'wearing', 24: 'yellow', 25: 'sign', 26: 'train', 27: 'sky', 28: 'wall', 29: 'water', 30: 'are', 31: 'building', 32: 'an', 33: 'table', 34: 'people', 35: 'standing', 36: 'large', 37: 'street', 38: 'tennis', 39: 'head', 40: 'sitting', 41: 'dog', 42: 'clock', 43: 'part', 44: 'bus', 45: 'small', 46: 'light', 47: 'plate', 48: 'cat', 49: 'holding', 50: 'to', 51: 'grass', 52: 'shirt', 53: 'window', 54: 'has', 55: 'at', 56: 'orange', 57: 'side', 58: 'ground', 59: 'boy', 60: 'giraffe', 61: 'plane', 62: 'bear', 63: 'girl', 64: 'car', 65: 'top', 66: 'playing', 67: 'road', 68: 'snow', 69: 'baseball', 70: 'horse', 71: 'field', 72: 'bench', 73: '

### Build tgt vocab

In [9]:
tgt_vocab = Vocabulary(tgt_min_word_freq, tgt_max_sent_len)            # Default Vocab
tgt_vocab.build_vocabulary(mt_data.df_train['Hindi Text Pre'])         # Building Vocabulary
mt_data.tgt_vocab = tgt_vocab
put_parm("tgt_vocab_size", tgt_vocab.vocab_size)

Total Tokens in the Dataset :  138669
Total Unique Tokens(Except pre-defined tokens) :  2621
{0: '<PAD>', 1: '<SOS>', 2: '<EOS>', 3: '<UNK>', 4: 'एक', 5: 'पर', 6: 'है', 7: 'के', 8: 'में', 9: 'का', 10: 'सफेद', 11: 'की', 12: 'आदमी', 13: 'और', 14: 'लाल', 15: 'साथ', 16: 'यह', 17: 'व्यक्ति', 18: 'हैं', 19: 'रंग', 20: 'महिला', 21: 'रहा', 22: 'दो', 23: 'सड़क', 24: 'से', 25: 'हुए', 26: 'काले', 27: 'काला', 28: 'ट्रेन', 29: 'दीवार', 30: 'पानी', 31: 'पहने', 32: 'आकाश', 33: 'नीला', 34: 'टेनिस', 35: 'बस', 36: 'भूरे', 37: 'बिल्ली', 38: 'हिस्सा', 39: 'घड़ी', 40: 'सिर', 41: 'लोग', 42: 'घास', 43: 'हुआ', 44: 'वाला', 45: 'प्लेट', 46: 'काली', 47: 'नीले', 48: 'हाथी', 49: 'को', 50: 'इमारत', 51: 'मेज', 52: 'लकड़ी', 53: 'हरे', 54: 'रही', 55: 'लड़की', 56: 'नारंगी', 57: 'खिड़की', 58: 'कर', 59: 'जिराफ', 60: 'खेल', 61: 'कार', 62: 'शर्ट', 63: 'कुत्ता', 64: 'जमीन', 65: 'बर्फ', 66: 'पीले', 67: 'खड़ा', 68: 'बेंच', 69: 'नीली', 70: 'पेड़', 71: 'विमान', 72: 'बेसबॉल', 73: 'लड़का', 74: 'हवा', 75: 'हरी', 76: 'किनारे', 77: 

In [10]:
print("Source Vocab Size : {}, Target Vocab Size : {}".format(get_parm("src_vocab_size"), get_parm("tgt_vocab_size")))

Source Vocab Size : 2214, Target Vocab Size : 2625


# Build Custom Dataset Class

In [11]:
class Custom_Dataset(Dataset):
    def __init__(self, df, src_vocab, tgt_vocab, p_type):
        super().__init__()
        self.df = df                                 # Dataset DataFrame 
        self.src_vocab = src_vocab                   # Source Vocabulary object
        self.tgt_vocab = tgt_vocab                   # Target Vocabulary object

        self.src_column = get_parm("src_column")          
        self.tgt_column = get_parm("tgt_column")                  
        
        self.src_sent = list(self.df[self.src_column])    # Source Sentences
        self.tgt_sent = list(self.df[self.tgt_column])    # Target Sentences

        self.src_encoding = self.src_vocab.sentence_indexing(self.src_sent).to(torch.int64) # Convert Source sentence to index list
        self.tgt_encoding = self.tgt_vocab.sentence_indexing(self.tgt_sent).to(torch.int64) # Convert Target sentence to index list
        
        
    def __len__(self):
        return len(self.df)                          # length of the dataset
    
    def __getitem__(self, idx):
        out_src = self.src_encoding[idx]             # Return Source Indexed sentence for the given index
        out_tgt = self.tgt_encoding[idx]             # Return Target Indexed sentence for the given index
        return out_src, out_tgt, idx

### Build Train, Val, Test Custom Dataset

In [12]:
train_data = Custom_Dataset(mt_data.df_train, src_vocab, tgt_vocab, 'train')  # Custom Train Data
val_data = Custom_Dataset(mt_data.df_val, src_vocab, tgt_vocab, 'val')        # Custom Val Data
test_data = Custom_Dataset(mt_data.df_test, src_vocab, tgt_vocab, 'test')     # Custom Test Data
challenge_data = Custom_Dataset(mt_data.df_challenge, src_vocab, tgt_vocab, 'challenge')    # Custom Test Data

### Build Train, Val, Test Dataloaders


In [13]:
batch_size = get_parm("batch_size")            # Batch_Size (512 is good)
num_workers = 4             # No of CPU cores to be used for getting the batch data (4 is most optimal)

train_dataloader = DataLoader(dataset = train_data, batch_size= batch_size, num_workers = num_workers, shuffle = True,  drop_last=True)
val_dataloader = DataLoader(dataset = val_data, batch_size= batch_size, num_workers = num_workers, shuffle = False)
test_dataloader = DataLoader(dataset = test_data, batch_size= batch_size, num_workers = num_workers, shuffle = False)
challenge_dataloader = DataLoader(dataset = challenge_data, batch_size= batch_size, num_workers = num_workers, shuffle = False)

dataloaders   = {"train": train_dataloader, "val": val_dataloader}

# Model Building

## Attention Mechanism

In [14]:
class Attention_Mechanism(nn.Module):
    def __init__(self):
        super(Attention_Mechanism, self).__init__()
        # Hyper-parameters
        self.embedding_dim = get_parm("dec_word_embd_dim")      # Source word embedding dimension
        self.hidden_dim = get_parm("dec_hidden_dim")            # RNN output dimension
        self.key_inp_dim = get_parm("enc_hidden_dim")*2 if get_parm("enc_bidirectional") \
                           else get_parm("enc_hidden_dim")      # Dimensions of Encoder hidden states
        
        # Attention Layers 
        self.key_proj = nn.Linear(self.key_inp_dim, self.hidden_dim, bias=False)    # Encoder outputs(key) layer
        self.query_proj = nn.Linear(self.hidden_dim, self.hidden_dim, bias=False)   # Decoder outputs(query) layer
        self.score_proj = nn.Linear(self.hidden_dim, 1)         # Alignment score calculation layer
        self.attn_softmax = nn.Softmax(dim=1)                   # Mapping alignment score to 0 to 1 
        
    def forward(self, enc_outputs,  hidden):
        
        key = self.key_proj(enc_outputs)                                   # Mapping Encoder Outputs into keys
        query = self.query_proj(hidden[0].view(hidden[0].shape[1],1,-1))   # Mapping Decoder Outputs into query     
        alignment_score = self.score_proj(torch.tanh(key + query)).squeeze(-1)  # Alignment Score b/w query & keys
        attn_prob = self.attn_softmax(alignment_score)     # Converting alignment score in the range of 0 to 1
        context_vector = torch.bmm(attn_prob.unsqueeze(1), enc_outputs)    # Context vector
        return context_vector, attn_prob
 

### Encoder

In [15]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.vocab_size = get_parm("src_vocab_size")            # Source Vocabulary Size
        self.padding_idx = 0                                    # Padding token index
        self.embedding_dim = get_parm("enc_word_embd_dim")      # Source word embedding dimension   
        self.embd_drop_prob = get_parm("enc_word_embd_dropout") # Embedding layer dropout probability
        
        self.embedding = nn.Embedding(num_embeddings= self.vocab_size, 
                                      embedding_dim = self.embedding_dim,
                                      padding_idx= self.padding_idx)         # Embedding Layer
        self.dropout = nn.Dropout(self.embd_drop_prob)          # Dropout Layer after Embedding Layer
        
        self.bidirectional_flag = True if get_parm("enc_bidirectional") else False   # Check if Bi-Direc LSTM use or not
        self.hidden_dim = get_parm("enc_hidden_dim")
        self.dec_hidden_dim = get_parm("dec_hidden_dim")
        self.rnn = nn.LSTM(self.embedding_dim, \
                           self.hidden_dim, \
                           batch_first = True, \
                           bidirectional = self.bidirectional_flag, \
                           dropout = get_parm("enc_rnn_dropout"))            # RNN

        self.device = get_parm("device")
        if self.bidirectional_flag:            
            self.hidden_init_dim = 2
        else:
            self.hidden_init_dim = 1
            
        print("***************************Encoder Hyper-parameters***************************")
        print("Input word one hot dim : ", self.vocab_size)
        print("Word Embedding dim : ", self.embedding_dim)
        print("Embedding Dropout : ", self.embd_drop_prob)
        print("Bidirectional : ", self.bidirectional_flag)
        print("RNN Hidden dim : ", self.hidden_dim)
        
        
    def forward(self, src_text):
        """
            function takes input sequence and initial hidden & cell state as input and returns the lstm outputs
           
            "src_text" contains input sequence. (batch_size, seq_len) (486, 13)
               [[  1,  11,  21,  ...,   0,   0,   2],
                [  1, 237,   8,  ...,   0,   0,   2],
                ...,
                [  1,  16, 143,  ...,   0,   0,   2]]

            Input sequence is passed through the embedding layer
            Embedding layer Output : (batch_size, seq_len, embedding_of_each_word) (486, 13, 256)
            Embedding output is passed through dropout layer
            
            "hidden" is a tuple contains : (initial_hidden_state, initial_cell_state)
            initial_hidden_state : (no_of_lstm_direction, batch_size, hidden_dim) (2, 486, 256)
            initial_cell_state   : (no_of_lstm_direction, batch_size, hidden_dim) (2, 486, 256)
            no_of_lstm_direction will be 1 if Bi-Directional is False else 2(Bi-Directional LSTM)
            These initial vectors are initialized by value 0 as there is no initial information.
            
            Embedding Output and initial_hidden are given as input to the LSTM
            LSTM Output :    hidden_state_of_each_timestamp, (last_hidden_state, last_cell_state)
            hidden_state_of_each_timestamp : (batch_size, seq_len, hidden_dim)  (486, 13, 512)
                 hidden_dim will be double of the defined lstm hidden_dim if Bi-directional LSTM is used
            
            For Bi-directional LSTM
            last_hidden_state : (no_of_lstm_direction, batch_size, hidden_dim) (2, 486, 256)
            last_cell_state   : (no_of_lstm_direction, batch_size, hidden_dim) (2, 486, 256)
            
            If decoder hidden_dim is equal to Bi-directional lstm hidden_dim,
                    then
                        (2, 486, 256) will be converted to (1, 486, 512) by concatenation
                    else        
                        (2, 486, 256) will be converted to (1, 486, 256) by addition
            2
            For Uni-directional LSTM
            last_hidden_state : (no_of_lstm_direction, batch_size, hidden_dim) (1, 486, 256)
            last_cell_state   : (no_of_lstm_direction, batch_size, hidden_dim) (1, 486, 256)
            
            
        """
        batch_size = src_text.shape[0]                              # Current Batch Size
        h_0, c_0 = self.init_hidden(batch_size)                     # Encoder's initial hidden and cell state
        
        embd_out = self.embedding(src_text)            # Passing the input sequence through embedding layer 
        embd_out_d = self.dropout(embd_out)            # Add dropout
        outputs, (h_t, c_t) = self.rnn(embd_out_d, (h_0, c_0))      # Pass text to LSTM
        if self.bidirectional_flag:
            if self.hidden_dim == self.dec_hidden_dim:        # Addition
                h_t = (h_t[0] + h_t[1]).unsqueeze(0)
                c_t = (c_t[0] + c_t[1]).unsqueeze(0)
            else:                                             # Concatenation
                h_t = torch.concat((h_t[0], h_t[1]), dim = 1).unsqueeze(0)
                c_t = torch.concat((c_t[0], c_t[1]), dim = 1).unsqueeze(0)
        
        return outputs, (h_t, c_t)
    
    def init_hidden(self, batch_size):
        h_0 = torch.zeros(self.hidden_init_dim, batch_size, self.hidden_dim, device = self.device)
        c_0 = torch.zeros(self.hidden_init_dim, batch_size, self.hidden_dim, device = self.device)
        return h_0, c_0


### Decoder

In [16]:
class Decoder(nn.Module):
    def __init__(self, use_attn = True):
        super(Decoder, self).__init__()
        # Hyper-parameters
        self.use_attn = use_attn
        self.vocab_size = get_parm("tgt_vocab_size")            # Source Vocabulary Size
        self.padding_idx = 0                                    # Padding token index
        self.embedding_dim = get_parm("dec_word_embd_dim")      # Source word embedding dimension
        self.hidden_dim = get_parm("dec_hidden_dim")            # RNN output dimension
        self.embd_drop_prob = get_parm("dec_word_embd_dropout") # Embedding layer dropout probability
        self.key_inp_dim = get_parm("enc_hidden_dim")*2 if get_parm("enc_bidirectional") \
                           else get_parm("enc_hidden_dim")      # Dimensions of Encoder hidden states
        
        # Embedding Layer
        self.embedding = nn.Embedding(num_embeddings= self.vocab_size, 
                                      embedding_dim = self.embedding_dim,
                                      padding_idx= self.padding_idx)    
        self.embd_dropout = nn.Dropout(self.embd_drop_prob)          # Dropout Layer after Embedding Layer
        
        
        
        if self.use_attn: 
            self.attention_text = Attention_Mechanism()              # Attention mechanism layer
            self.decoder_inp_dim = self.key_inp_dim + self.embedding_dim
        else:
            self.decoder_inp_dim = self.embedding_dim
        
        # Decoder LSTM
        self.rnn = nn.LSTM(self.decoder_inp_dim, 
                           self.hidden_dim, 
                           batch_first = True, 
                           bidirectional = False, 
                           dropout = get_parm("dec_rnn_dropout"))   # RNN
        
        # Output word classification layer
        self.classifier = nn.Linear(self.hidden_dim, self.vocab_size)   # Linear layer for word classification
        self.classifier_softmax = nn.LogSoftmax(dim = 2)       # Softmax layer for NLLLoss, Remove if using Cross_Entroy
        
        print("\n\n***************************Decoder Hyper-parameters***************************")
        print("Input word one hot dim : ", self.vocab_size)
        print("Word Embedding dim : ", self.embedding_dim)
        print("Embedding Dropout : ", self.embd_drop_prob)
        print("RNN Hidden dim : ", self.hidden_dim)
        print("Using Attention Mechanism : ", self.use_attn)
        
    def forward(self, gen_word, enc_outputs , hidden):
        """
            function takes previously generated word and encoder output. Returns the current predicted
                       word and decoder's current hidden state
                       
            **** Sequence length will be only 1 as single word is generated at a time
            
            "gen_word" contains input sequence. (batch_size, 1) (486, 1)
            Initally <SOS> will be input whose index is 1.
              Ex. 1 :   [[1],[1],[1],[1],[1], ............ [1],[1],[1],[1]]   
            Then previously generated word will be given as input
              Ex. 2 :   [[5717],[3267],[6075],[6308], ............ [3709],[3725],[3725],[1118]]
               
            Input sequence is passed through the embedding layer
            Embedding layer Output : (batch_size, seq_len, embedding_of_each_word) (486, 1, 256)
            Embedding output is passed through dropout layer
            
            "hidden" is a tuple contains : (hidden_state, cell_state)
            hidden_state : (no_of_lstm_direction, batch_size, hidden_dim) (1, 486, 256)
            cell_state   : (no_of_lstm_direction, batch_size, hidden_dim) (1, 486, 256)
            
            Initially Encoder's hidden and cell state will be given as input. 
            After that Decoder's hidden and cell state will be given as input.
            
            Embedding Output and initial_hidden are given as input to the LSTM
            LSTM Output :    hidden_state_of_each_timestamp, (last_hidden_state, last_cell_state)
            hidden_state_of_each_timestamp : (batch_size, seq_len, hidden_dim)  (486, 1, 256)
            
            last_hidden_state : (no_of_lstm_direction, batch_size, hidden_dim) (1, 486, 256)
            last_cell_state   : (no_of_lstm_direction, batch_size, hidden_dim) (1, 486, 256)
            
            LSTM Output will be fed to a Linear classification and softmax layer,
            which will predict the probability for the output word.
            Linear layer contains neuron equal to the target vocabulary size.
            
            Linear layer Output : (batch_size, seq_len, tgt_vocab_size)  (486, 1, 6941)
            Linear layer Output the logits which needs to be converted into probabilities.
            Linear layer output is passed to the logsoftmax layer which converts logits into the range of (-inf,0).
            
        """
        
        
        embd_out = self.embedding(gen_word) 
        embd_out = self.embd_dropout(embd_out)
        if self.use_attn:
            context_vector_text, attn_prob_text = self.attention_text(enc_outputs, hidden)
            #context_vector_text = F.relu(context_vector_text)
            outputs, hidden = self.rnn(torch.concat((embd_out, context_vector_text),dim = 2), hidden)
        else:
            attn_prob_text = []
            outputs, hidden = self.rnn(embd_out, hidden)
            
        predictions = self.classifier(outputs)
        predictions = self.classifier_softmax(predictions)
        return predictions, hidden, attn_prob_text

### Encoder-Decoder Model

In [17]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.device = get_parm("device")                    # Getting the device
        self.encoder = encoder                              # Encoder
        self.decoder = decoder                              # Decoder
        self.target_len = get_parm("tgt_max_sent_len")+2    # Length of target sequence
        self.tgt_vocab_size = self.decoder.vocab_size       # target vocab size
        self.dec_initial_input = torch.ones(get_parm("batch_size"),1, dtype=torch.int, device = self.device)
        
    def forward(self, src):
        batch_size = src.shape[0]
        enc_outputs, hidden = self.encoder(src)       # Passing input to the encoder
        dec_inp = self.dec_initial_input[:batch_size,:]           # Deocder's initial input : <SOS>, Index : 1
        
        # Output Matrix (batch_size, seq_len, tgt_vocab_size). Last dim will contain output probabilities for each timestamp
        output = torch.zeros(batch_size, self.target_len, self.tgt_vocab_size, device = self.device)
        attn_list = []
        # Iterating for the output sequence, <EOS> will not be given, so target sequence len will be reduced by 1
        for t in range(1, self.target_len):
            predictions, hidden, attn = self.decoder(dec_inp, enc_outputs, hidden)   # Generating current word prediction
            dec_inp = torch.argmax(predictions.squeeze(), dim = 1).unsqueeze(dim=1)   # Getting output word with max probability
            output[:,t] = predictions.squeeze()       # Adding output probabilities to the output matrix
            attn_list.append(attn)
        predictions, hidden, attn = self.decoder(dec_inp, enc_outputs, hidden)
        return output, attn_list

# Build Model

In [18]:
def build_model():
    device = get_parm("device")
    encoder = Encoder()
    decoder = Decoder()
    model = Seq2Seq(encoder, decoder).to(device)
    
    layer_count = 1
    print("Layers in Model : ")
    for name,param in model.named_parameters():
        print("\tLayer {} : {}, Requires_Grad : {}".format(layer_count, name, param.requires_grad))
        layer_count += 1
    
    learning_rate = 0.001
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
    criterion = nn.NLLLoss(ignore_index=0)
    criterion = criterion.to(device) 
    print('\n\n')
    print(model)
    return model, criterion, optimizer

model, criterion, optimizer = build_model()

***************************Encoder Hyper-parameters***************************
Input word one hot dim :  2214
Word Embedding dim :  256
Embedding Dropout :  0.3
Bidirectional :  True
RNN Hidden dim :  256


***************************Decoder Hyper-parameters***************************
Input word one hot dim :  2625
Word Embedding dim :  256
Embedding Dropout :  0.3
RNN Hidden dim :  256
Using Attention Mechanism :  True
Layers in Model : 
	Layer 1 : encoder.embedding.weight, Requires_Grad : True
	Layer 2 : encoder.rnn.weight_ih_l0, Requires_Grad : True
	Layer 3 : encoder.rnn.weight_hh_l0, Requires_Grad : True
	Layer 4 : encoder.rnn.bias_ih_l0, Requires_Grad : True
	Layer 5 : encoder.rnn.bias_hh_l0, Requires_Grad : True
	Layer 6 : encoder.rnn.weight_ih_l0_reverse, Requires_Grad : True
	Layer 7 : encoder.rnn.weight_hh_l0_reverse, Requires_Grad : True
	Layer 8 : encoder.rnn.bias_ih_l0_reverse, Requires_Grad : True
	Layer 9 : encoder.rnn.bias_hh_l0_reverse, Requires_Grad : True
	Layer 10 :

# Training Model

In [19]:
def train_model(model, criterion, optimizer, dataloaders, mt_data, num_epochs= 1000):
    since = time.time()
    device = get_parm("device")
    best_model_wts = copy.deepcopy(model.state_dict())
    best_bleu = 0.0
    best_epoch = 0
    patience = 30
    patience_counter = 0
    break_flag = False
    file_dir = 'Result'
    train_loss_list = []
    val_loss_list = []
    bleu_main = []
    bleu_comb = []
    scaler = torch.cuda.amp.GradScaler() 
    for epoch in tqdm(range(num_epochs)):
        patience = get_parm("patience")
        print("*"*50)
        print(f'Epoch {epoch}/{num_epochs - 1}', end = "    ")
        #print("-"*10)
        
        for phase in ['train', 'val']: # We do training and validation phase per epoch
            if phase == 'train':
                model.train() # model to training mode
                start = time.time()
            else:
                model.eval() # model to evaluate
                start = time.time()
            
            batch_loss = 0.0 
            batch_count = 0
            src_raw_english = []
            src_model_inp_english = []
            tgt_raw_hindi = []
            tgt_pred_hindi = []
            for src, tgt, idx in dataloaders[phase]:
                #since = time.time()
                src, tgt = src.to(device), tgt.to(device)
                optimizer.zero_grad()
                time_elapsed = time.time() - since 
                #print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
                if phase == 'train':
                    with autocast(device_type=device, dtype=torch.float16):
                        outputs, attn = model(src)
                        loss = criterion(outputs[:,1:].reshape(-1, outputs.shape[2]), tgt[:,1:].reshape(-1))
                    scaler.scale(loss).backward()
                    
                    # Unscales the gradients of optimizer's assigned params in-place
                    scaler.unscale_(optimizer)

                    # Since the gradients of optimizer's assigned params are unscaled, clips as usual:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 10)

                    # update the scaled parameters
                    scaler.step(optimizer)
                    # update the scaler for next iteration
                    scaler.update()
                    
                if phase == 'val':
                    p_type = 'val'
                    with torch.no_grad():
                        outputs, attn = model(src)
                    src_english, inp_english, tgt_hindi, pred_hindi = mt_data.output_to_sentence_val(idx, src, outputs, p_type)
                    src_raw_english.extend(src_english)
                    src_model_inp_english.extend(inp_english)
                    tgt_raw_hindi.extend(tgt_hindi)
                    tgt_pred_hindi.extend(pred_hindi)


                batch_loss += loss.item()
                batch_count += 1
               
            if phase == 'val':
                p_type = 'val'
                with open('./{}/{}_src_english.txt'.format(file_dir,p_type),'w') as f:
                    f.writelines(src_raw_english)
                with open('./{}/{}_model_inp_english.txt'.format(file_dir,p_type),'w') as f:
                    f.writelines(src_model_inp_english)
                with open('./{}/{}_tgt_hindi.txt'.format(file_dir,p_type),'w') as f:
                    f.writelines(tgt_raw_hindi)
                with open('./{}/{}_pred_hindi.txt'.format(file_dir,p_type),'w') as f:
                    f.writelines(tgt_pred_hindi)

                script = "perl multi-bleu.perl  ./{}/{}_tgt_hindi.txt < ./{}/{}_pred_hindi.txt".format(file_dir,p_type,file_dir,p_type)
    
                bleu_score_out = subprocess.getoutput(script)
                print(bleu_score_out)
                if 'uninitialized' in bleu_score_out:
                    bleu_main.append(0.0)
                    bleu_comb.append('')
                    epoch_bleu = 0.0
                else:
                    bleu_score_out = bleu_score_out.split()
                    #print(bleu_score_out)
                    try:
                        epoch_bleu = float(bleu_score_out[2].split(',')[0])
                    except:
                        epoch_bleu = 0.0
                    bleu_main.append(epoch_bleu)
                    bleu_comb.append(bleu_score_out[3])
            
            epoch_loss = batch_loss / batch_count
            if phase == 'train':
                print("{} Loss: {:.4f} ".format(phase, epoch_loss))
            else:
                print("{} Loss: {:.4f} bleu score: {:.4f}".format(phase, epoch_loss, epoch_bleu))
            
            if phase == 'val':
                val_loss_list.append(epoch_loss)
                
                if epoch_bleu >= best_bleu:
                    print("Got better bleu score")
                    patience_counter = 0
                    best_epoch = epoch
                    print("Best Epoch till now : ",best_epoch)
                    best_bleu = epoch_bleu
                    best_model_wts = copy.deepcopy(model.state_dict()) # keep the best validation accuracy model
                else:
                    print("Got lower bleu score")
                    print("Curr bleu score {}, prev best bleu score {}".format(epoch_bleu, best_bleu))
                    patience_counter += 1
                    if patience_counter >= patience:
                        print("Model is not improving from previous {} epochs. Stopping Training".format(patience))
                        print("Best Epoch : {}".format(best_epoch))
                        print("Best BLEU : {}".format(best_bleu))
                        break_flag = True
                        break
                print("Patience : ",patience_counter)
                end = time.time()
                print('Time Taken in completing Val step : %.6f' % (end - start))
            else:
                train_loss_list.append(epoch_loss)
                end = time.time()
                print('Time Taken in completing Train step : %.6f' % (end - start))
            
        if break_flag:
            break
    time_elapsed = time.time() - since 
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    metrics = {"bleu_main":bleu_main, "bleu_comb":bleu_comb, \
                            "Train_Loss":train_loss_list, "Val_Loss":val_loss_list}
    model.load_state_dict(best_model_wts)
                
    os.remove('./{}/{}_src_english.txt'.format(file_dir,p_type))
    os.remove('./{}/{}_model_inp_english.txt'.format(file_dir,p_type))
    os.remove('./{}/{}_tgt_hindi.txt'.format(file_dir,p_type))
    os.remove('./{}/{}_pred_hindi.txt'.format(file_dir,p_type))
    
    return model, metrics

In [20]:
if get_parm("pre_trained"):
    model.load_state_dict(torch.load(get_parm("pre_trained_model")))
    pytorch_total_params = sum(p.numel() for p in model.parameters())
    print("Total Parameters in Model : ",pytorch_total_params)
    model.eval()

else:
    model, metrics = train_model(model, criterion, optimizer, dataloaders, mt_data)
    now = datetime.now()
    dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")
    torch.save(model, "./Weights_Metrics/"+'_' + 'Complete_Model_' + dt_string + '.pt')
    torch.save(model.state_dict(), "./Weights_Metrics/"+'_' + 'Model_Weights_' + dt_string + '.pt')
    
    bleu_main = [i for i in metrics["bleu_main"]]
    bleu_comb = [i for i in metrics["bleu_comb"]]
    train_loss_list = [float(i) for i in metrics["Train_Loss"]]
    val_loss_list = [float(i) for i in metrics["Val_Loss"]]
    df = pd.DataFrame({"bleu_main":bleu_main, "bleu_comb":bleu_comb,"Train_Loss": train_loss_list,"Val_Loss" : val_loss_list })
    df.to_csv("./Weights_Metrics/"+'_'+'Train_metrics_' + dt_string + '.csv',index=None)


  0%|                                                                                                                                   | 0/1000 [00:00<?, ?it/s]

**************************************************
Epoch 0/999    train Loss: 4.7514 
Time Taken in completing Train step : 1.614532


  0%|                                                                                                                           | 1/1000 [00:01<30:33,  1.84s/it]

BLEU = 1.80, 31.6/6.3/0.4/0.2 (BP=0.888, ratio=0.894, hyp_len=4423, ref_len=4950)
val Loss: 4.0839 bleu score: 1.8000
Got better bleu score
Best Epoch till now :  0
Patience :  0
Time Taken in completing Val step : 0.220790
**************************************************
Epoch 1/999    train Loss: 3.7409 
Time Taken in completing Train step : 1.305937


  0%|▏                                                                                                                          | 2/1000 [00:03<27:28,  1.65s/it]

...

BLEU = 35.88, 65.6/43.5/29.2/19.9 (BP=1.000, ratio=1.001, hyp_len=4957, ref_len=4950)
val Loss: 0.1680 bleu score: 35.8800
Got lower bleu score
Curr bleu score 35.88, prev best bleu score 37.3
Patience :  48
Time Taken in completing Val step : 0.234153
**************************************************
Epoch 165/999    train Loss: 0.1229 
Time Taken in completing Train step : 1.289106


 17%|████████████████████                                                                                                     | 166/1000 [04:10<20:48,  1.50s/it]

BLEU = 36.50, 65.8/43.9/29.8/20.6 (BP=1.000, ratio=1.003, hyp_len=4966, ref_len=4950)
val Loss: 0.1708 bleu score: 36.5000
Got lower bleu score
Curr bleu score 36.5, prev best bleu score 37.3
Patience :  49
Time Taken in completing Val step : 0.239636
**************************************************
Epoch 166/999    train Loss: 0.1273 
Time Taken in completing Train step : 1.312921


 17%|████████████████████                                                                                                     | 166/1000 [04:12<21:07,  1.52s/it]

BLEU = 36.88, 66.0/44.4/30.2/20.9 (BP=1.000, ratio=1.000, hyp_len=4948, ref_len=4950)
val Loss: 0.1463 bleu score: 36.8800
Got lower bleu score
Curr bleu score 36.88, prev best bleu score 37.3
Model is not improving from previous 50 epochs. Stopping Training
Best Epoch : 116
Best BLEU : 37.3
Training complete in 4m 12s


# Inference

In [24]:
def test_results(model, test_dataloader, p_type):
    model.eval()
    file_dir = 'Result'
    src_raw_english = []
    src_model_inp_english = []
    tgt_raw_hindi = []
    tgt_pred_hindi = []
    all_results = []
    for src, tgt, idx in test_dataloader:
        src = src.to(device)
        with torch.set_grad_enabled(False): # no autograd makes validation go faster
            outputs, attn = model(src)
            src_english, inp_english, tgt_hindi, pred_hindi = mt_data.output_to_sentence_val(idx, src, outputs, p_type)
            src_raw_english.extend(src_english)
            src_model_inp_english.extend(inp_english)
            tgt_raw_hindi.extend(tgt_hindi)
            tgt_pred_hindi.extend(pred_hindi)
            all_results.append([src_english, inp_english, tgt_hindi, pred_hindi, attn])

            
    with open('./{}/{}_src_english.txt'.format(file_dir,p_type),'w') as f:
        f.writelines(src_raw_english)
    with open('./{}/{}_model_inp_english.txt'.format(file_dir,p_type),'w') as f:
        f.writelines(src_model_inp_english)
    with open('./{}/{}_tgt_hindi.txt'.format(file_dir,p_type),'w') as f:
        f.writelines(tgt_raw_hindi)
    with open('./{}/{}_pred_hindi.txt'.format(file_dir,p_type),'w') as f:
        f.writelines(tgt_pred_hindi)
            

    script = "perl multi-bleu.perl  ./{}/{}_tgt_hindi.txt < ./{}/{}_pred_hindi.txt".format(file_dir,p_type,file_dir,p_type)
    bleu_score_out = subprocess.getoutput(script)
    print(bleu_score_out)
    index_attn = len(src_raw_english)
    for i in range(len(src_raw_english)):
        print("**************************************************")
        print(index_attn)
        print("Source English : ",src_raw_english[i])
        print("Input English : ",src_model_inp_english[i])
        print("Original Hindi : ",tgt_raw_hindi[i])
        print("Predicted Hindi : ", tgt_pred_hindi[i])    
        index_attn -= 1
    return all_results

# Test Result

In [25]:
print("Test")
d_ld = test_dataloader
p_type = 'test'
attn_test = test_results(model, d_ld, p_type)

Test
BLEU = 34.21, 64.2/42.3/28.1/18.0 (BP=1.000, ratio=1.005, hyp_len=7819, ref_len=7780)
**************************************************
1595
Source English :  the orange colored traffic cone

Input English :  <SOS> the orange colored traffic cone <EOS>

Original Hindi :  नारंगी रंग यातायात शंकु

Predicted Hindi :  नारंगी रंग का ट्रैफिक शंकु

**************************************************
1594
Source English :  a stop light

Input English :  <SOS> a stop light <EOS>

Original Hindi :  एक स्टॉप लाइट

Predicted Hindi :  एक स्टॉप लाइट

**************************************************
1593
Source English :  three zebras in the wild

Input English :  <SOS> three zebras in the wild <EOS>

Original Hindi :  जंगल में तीन जेब्रा

Predicted Hindi :  जंगली में तीन ज़ेबरा

**************************************************
1592
Source English :  cars parked along the side of the road

Input English :  <SOS> cars parked along the side of the road <EOS>

Original Hindi :  सड़क के किनारे ख

# Challenge Result

In [26]:
print("Challenge")
d_ld = challenge_dataloader
p_type = 'challenge'
attn_chlng = test_results(model, d_ld, p_type)

Challenge
BLEU = 20.19, 52.3/28.9/16.3/9.2 (BP=0.927, ratio=0.929, hyp_len=8011, ref_len=8622)
**************************************************
1400
Source English :  a bunch of books on book stand

Input English :  <SOS> a bunch of books on book stand <EOS>

Original Hindi :  पुस्तक स्टैंड पर पुस्तकों का एक गुच्छा

Predicted Hindi :  शीर्ष स्टैंड पर <UNK> का एक गुच्छा

**************************************************
1399
Source English :  gold religious cross on top of golden ball

Input English :  <SOS> gold <UNK> cross on top of golden ball <EOS>

Original Hindi :  स्वर्ण गेंद के शीर्ष पर स्वर्ण धार्मिक क्रॉस

Predicted Hindi :  सलाद गेंद के शीर्ष पर सोने का <UNK>

**************************************************
1398
Source English :  candle on glass candle stand

Input English :  <SOS> candle on glass candle stand <EOS>

Original Hindi :  कांच के मोमबत्ती स्टैंड पर मोमबत्ती

Predicted Hindi :  कांच की स्टैंड पर मोमबत्ती

**************************************************
13

# Generating AttentionMap

In [27]:
shutil.copy('./Nirmala.ttf', '../anaconda3/envs/mt/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf')
hindi_font = FontProperties(fname = '../anaconda3/envs/mt/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/Nirmala.ttf')
english_font = FontProperties(fname = '../anaconda3/envs/mt/lib/python3.11/site-packages/matplotlib/mpl-data/fonts/ttf/DejaVuSans.ttf')

from matplotlib import font_manager as fm
def plot_heatmaps(xticklabels, yticklabels, attn_probs, sent_idx, p_type):
    x_len = len(xticklabels)
    y_len = len(yticklabels)
    attn_probs = np.array(attn_probs)
    fig, ax = plt.subplots(figsize=(x_len,y_len))
    im = ax.imshow(attn_probs,cmap='gist_gray')
    

    
    # Show all ticks and label them with the respective list entries
    ax.set_xticks(np.arange(len(xticklabels)), labels=xticklabels, font = english_font, fontsize = 16 ,rotation = 90)
    ax.set_yticks(np.arange(len(yticklabels)), labels=yticklabels, font = hindi_font, fontsize = 16 )

    # Loop over data dimensions and create text annotations.
    
    for i in range(len(yticklabels)):
        for j in range(len(xticklabels)):
            text = ax.text(j, i, attn_probs[i][j],
                           ha="center", va="center", color="black", fontsize = 16)
    # Show the top tick labels
    ax.xaxis.set_tick_params(labeltop=True)
    plt.tick_params(bottom = False)
    plt.tick_params(top = True)

    # Hide the bottom tick labels
    ax.xaxis.set_tick_params(labelbottom=False)
    fig.tight_layout()
    #print('./Attention_Maps/{}_{}_{}.png'.format(p_type, sent_idx, get_parm('approach_type')))
    plt.savefig('./Attention_Maps/{}_{}.png'.format(p_type, sent_idx),dpi= 300)
    plt.close()
    #plt.show()

### Attention Heatmaps

In [30]:
def process_attention(attn_test, p_type):
    sent_idx = 1
    for batch_idx in range(len(attn_test)):
        src_english, inp_english, src_hindi, pred_hindi, attn = attn_test[batch_idx]
        for i in tqdm(range(len(src_english))):
#             print("Input Sentence : ",inp_english[i])
#             print("Output Sentence : ",pred_hindi[i])
            inp_word_list = re.sub('\n','',inp_english[i]).split()
            gen_word_list = re.sub('\n','',pred_hindi[i]).split()
            attn_probs = []
            for j in range(len(gen_word_list)):
                #print(attn[j][i])
                
                try:
                    focus_idx = torch.argmax(attn[j][i], dim=0).item()
                    #print("{} : {}".format(gen_word_list[j],inp_word_list[focus_idx]), end = ', ')
                except:
                    focus_word = "<PAD>"
                focus_prob = attn[j][i][focus_idx].item()

                try:
                    focus_word = inp_word_list[focus_idx]
                except:
                    focus_word = "<PAD>"
                    #print("Focused on PAD token. Sentence ID : ",sent_idx)
                    #print("Sentence : ", pred_hindi[i])
                prob_list = list(attn[j][i].cpu().numpy())[1:len(inp_word_list)-1]
                prob_list = [round(num, 2) for num in prob_list]
                #print("Attention Probs : ", prob_list)
                attn_probs.append(prob_list)
            #print('')
            plot_heatmaps(inp_word_list[1:-1], gen_word_list, attn_probs, sent_idx, p_type)

            sent_idx += 1


In [31]:
process_attention(attn_test, 'test')

 12%|███████████████▍                                                                                                           | 32/256 [00:04<00:35,  6.36it/s]/tmp/ipykernel_114721/4262413523.py:32: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all axes decorations.
  fig.tight_layout()
 59%|███████████████████████████████████████████████████████████████████████▉                                                  | 151/256 [00:18<00:10,  9.98it/s]/tmp/ipykernel_114721/4262413523.py:32: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all axes decorations.
  fig.tight_layout()
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 59/59 [00:07<00:00,  8.17it/s]
